| Concept           | Frequency Range   | Notes / MIDI Range      | Notes                        |
| ----------------- | ----------------- | ----------------------- | ---------------------------- |
| Audible (human)   | 20 Hz – 20 kHz    | MIDI 0 – 127            | Full MIDI                    |
| Playful / Musical | 27.5 Hz – 4186 Hz | MIDI 21 (A0) – 108 (C8) | Piano & general instruments  |
| Safe / Harmonics  | 80 Hz – 12 kHz    | MIDI ~28 – 110          | Most instruments and effects |


In [1]:
print("setup seccessful")

setup seccessful


### MIDI Note Numbers to Frequency

The formula to convert a MIDI note number to frequency (Hz) is:

$$
f = 440 \times 2^{\frac{(n - 69)}{12}}
$$

Where:
- \( f \) = frequency in Hz
- \( n \) = MIDI note number
- 440 Hz = standard tuning for A4 (MIDI note 69)

**Example:**
- Middle C (MIDI 60):
$$
f = 440 \times 2^{\frac{60 - 69}{12}} \approx 261.63 \text{ Hz}
$$

---

### Frequency to MIDI Note Number

To find the MIDI note number from a frequency:

$$
n = 69 + 12 \times \log_2\left(\frac{f}{440}\right)
$$

**Example:**
- Frequency 523.25 Hz (C5):
$$
n = 69 + 12 \times \log_2\left(\frac{523.25}{440}\right) \approx 72
$$

---

### MIDI Note Number to Musical Note

A common mapping for note names:

```text
0 = C-1
1 = C#-1 / Db-1
2 = D-1
...
60 = C4 (Middle C)
69 = A4


In [ ]:
# List of note names in an octave
note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

print(f"{'MIDI':<5} {'Note':<4}")
print('-' * 10)

# MIDI note numbers typically range from 0 to 127
for midi_num in range(69,128):
    octave = (midi_num // 12) - 1      # MIDI standard: C4 = 60
    note = note_names[midi_num % 12]
    print(f"{midi_num:<5} {note}{octave:<3}")


In [ ]:
!pip install mido pygame

In [ ]:
from mido import Message, MidiFile, MidiTrack


## 1. Beats

In [ ]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

KICK = 36; SNARE = 38; HIHAT = 42
ticks = 480

pattern = [
    (KICK, 0),
    (HIHAT, 0),
    (SNARE, ticks),
    (HIHAT, 0),
    (KICK, ticks),
    (HIHAT, 0),
    (SNARE, ticks),
    (HIHAT, 0)
]

for note, delay in pattern:
    track.append(Message('note_on', note=note, velocity=100, time=delay))
    track.append(Message('note_off', note=note, velocity=100, time=120))

mid.save("midi-clips/beat.mid")


## 2. Harmony Clip (Chord Progression)

In [ ]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

chords = [
    [60,64,67],   # C
    [67,71,74],   # G
    [69,72,76],   # Am
    [65,69,72],   # F
]

for chord in chords:
    for n in chord:
        track.append(Message('note_on', note=n, velocity=80, time=0))
    for n in chord:
        track.append(Message('note_off', note=n, velocity=80, time=960))

mid.save("midi-clips/harmony.mid")


## 3. Melody Clip (Long Single Clip)

In [ ]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

melody_notes = [60, 62, 64, 67, 69, 67, 64, 62] * 2  # 16-bar idea
time_step = 360

for note in melody_notes:
    track.append(Message('note_on', note=note, velocity=90, time=time_step))
    track.append(Message('note_off', note=note, velocity=90, time=240))

mid.save("midi-clips/melody.mid")


## 4. Play MIDI with pygame

In [3]:

!pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.6 MB 3.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/10.6 MB 3.0 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/10.6 MB 3.4 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/10.6 MB 3.6 MB/s eta 0:00:03
   -------------- ------------------------- 3.9/10.6 MB 3.7 MB/s eta 0:00:02
   ----------------- ---------------------- 4.7/10.6 MB 3.7 MB/s eta 0:00:02
   --------------------- ------------------ 5.8/10.6 MB 4.0 MB/s eta 0:00:02
   ------------------------- -------------- 6.8/10.6 MB 4.1 MB/s eta 0:00:01
   ----------------------------- ---------- 7.9/10.6 MB 4.3 MB/s eta 0:00:01
   ---------------------------------- ----- 9.2/10.6 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------  10.5/10.6 MB 4.6 MB/s eta 0:00:01
   ---------

In [4]:
import pygame

pygame.mixer.init()
pygame.mixer.music.load("midi-clips/melody.mid")
pygame.mixer.music.play()

print("Playing MIDI...")
while pygame.mixer.music.get_busy():
    pass


c:\Users\ACER\.conda\envs\venv\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html
Playing MIDI...


## 5. MIDI to Audio 

In [5]:
import os
import subprocess

sf = os.path.expanduser("~/Downloads/FluidR3GM2-2.SF2")

clip = "melody"

midi = "midi-clips/"+clip+".mid"
out = "audio-clips/"+clip+".wav"

cmd = [
    "fluidsynth",
    "-n",             # no audio driver
    "-i",             # no shell
    "-F", out,        # write to file
    "-r", "44100",    # sample rate
    sf,               # SoundFont
    midi              # MIDI file
]

print("Running:", " ".join(cmd))
subprocess.run(cmd)
print("Saved:", out)


Running: fluidsynth -n -i -F audio-clips/melody.wav -r 44100 C:\Users\ACER/Downloads/FluidR3GM2-2.SF2 midi-clips/melody.mid


FileNotFoundError: [WinError 2] The system cannot find the file specified

## PyDub

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

# Load pre-rendered clips
beat = AudioSegment.from_wav("audio-clips/beat.wav")          # short loop
harmony = AudioSegment.from_wav("audio-clips/harmony.wav")    # short loop
melody = AudioSegment.from_wav("audio-clips/melody.wav")      # long single clip

# Repeat beat & harmony to match melody length
repeat_count = int(len(melody) / len(beat)) + 1
beat_loop = beat * repeat_count
harmony_loop = harmony * repeat_count

# Trim loops to melody length
beat_loop = beat_loop[:len(melody)]
harmony_loop = harmony_loop[:len(melody)]

# Mix all tracks
final_mix = beat_loop.overlay(harmony_loop).overlay(melody)

# Play
# play(final_mix)

# export
final_mix.export("audio-clips/final_song_pydub.wav", format="wav")


In [ ]:
# !pip install midi2audio
# ! pip install pygame

## Librosa

In [ ]:
import librosa
import soundfile as sf
import numpy as np
from IPython.display import Audio

# Load audio as numpy arrays
beat, sr = librosa.load("audio-clips/beat.wav", sr=None)
harmony, _ = librosa.load("audio-clips/harmony.wav", sr=None)
melody, _ = librosa.load("audio-clips/melody.wav", sr=None)

# Repeat beat & harmony to match melody length
repeat_count = int(len(melody) / len(beat)) + 1
beat_loop = np.tile(beat, repeat_count)[:len(melody)]
harmony_loop = np.tile(harmony, repeat_count)[:len(melody)]

# Mix tracks
final_mix = beat_loop + harmony_loop + melody

# Normalize to prevent clipping
final_mix = final_mix / np.max(np.abs(final_mix))

# Save final audio
sf.write("audio-clips/final_song_librosa.wav", final_mix, sr)

# Play in notebook
Audio(final_mix, rate=sr)


: 